In [111]:
import os
from pathlib import Path

import imageio.v2 as imageio
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np


def normalize(img):
    """Normalize the image for better visualization."""
    img = img - np.min(img)
    img = img / np.max(img)
    return img

def padded_image(data, max_dimension):
    """Pad the image to make it square."""
    padded = np.zeros((max_dimension, max_dimension))
    x_pad = round((max_dimension - data.shape[0])/2)
    y_pad = round((max_dimension - data.shape[1])/2)
    padded[x_pad:x_pad+data.shape[0], y_pad:y_pad+data.shape[1]] = data
    return padded

def load_nifti_slice(file_path, slice_index):
        """Load a NIfTI file and extract specific slices."""
        img = nib.load(file_path)
        data = img.get_fdata()
        data = normalize(data)

        max_dimension = max(data.shape)
        axial = padded_image(data[:, :, 98].copy(), max_dimension).T
        sagittal = padded_image(data[98, :, :].copy(), max_dimension).T
        coronal = padded_image(data[:, 98, :].copy(), max_dimension).T
        
        return axial, sagittal, coronal

def save_annotated_image(data, annotation, output_path):
        """Save a slice with annotation as an image."""
        plt.imshow(data, cmap="gray", origin="lower")
        plt.title(annotation)
        plt.axis("off")
        plt.savefig(output_path, bbox_inches="tight", pad_inches=0)
        plt.close()

def make_gif(out_file, files, *, slice_index=98):
    # Directory to save temporary images
    temp_dir = Path("temp_images")
    temp_dir.mkdir(exist_ok=True)

    # List to store paths of the saved images
    image_paths: list[Path] = []

    # Load, annotate, and save each slice
    for i, (annotation, file_path) in enumerate(files.items()):
        # Change slice_index as needed
        axial, sagittal, coronal = load_nifti_slice(file_path, slice_index)
        combined_planes = np.hstack((axial, sagittal, coronal))
        image_path = temp_dir / f"image_{i}.png"
        save_annotated_image(combined_planes, annotation, image_path)
        image_paths.append(image_path)

    # Create GIF
    images = [imageio.imread(image_path.as_posix()) for image_path in image_paths]
    for image in images:
         print(image.shape)
    imageio.mimsave(out_file, images, fps=2, loop=0)

    # Clean up temporary images
    for image_path in image_paths:
        os.remove(image_path)
    temp_dir.rmdir()

# All

In [112]:
from tqdm import tqdm

warped_images = list(Path("dataset/ds004513/derivatives/flint/antsRegistration").rglob("Warped.nii.gz"))
template = Path("tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz")
output_dir = Path("gifs/vprec")

for image in tqdm(warped_images):
    exp_id = image.parent.parent.name
    subject_id = image.parent.name
    output_file = output_dir / exp_id / f"{subject_id}.gif"
    output_file.parent.mkdir(exist_ok=True, parents=True)

    input_files = {
        f"{exp_id}: {subject_id}": image,
        "template": template,
    }
    make_gif(output_file, input_files)


  0%|          | 1/500 [00:07<59:00,  7.09s/it]

(187, 496, 4)
(187, 496, 4)


  0%|          | 2/500 [00:11<46:59,  5.66s/it]

(187, 496, 4)
(187, 496, 4)


  1%|          | 3/500 [00:15<39:23,  4.76s/it]

...

(187, 496, 4)
(187, 496, 4)


100%|█████████▉| 499/500 [17:17<00:02,  2.73s/it]

(187, 496, 4)
(187, 496, 4)


100%|██████████| 500/500 [17:20<00:00,  2.08s/it]

(187, 496, 4)
(187, 496, 4)
